# Sentiment Analysis

Obtain textual data for sentiment analysis from financial news sources and social media. Sentiment Analysis: Utilize GPT api to run sentiment analysis on the textual data to extract sentiment scores for each REIT in the office sector.

In [1]:
# import and setup necessary libraries
import pandas as pd
# import openai
# openai.organization = "YOUR_ORG_ID"
# openai.api_key = os.getenv("API_KEY")
# openai.Model.list()

In [41]:
# Load the CSV file
data = pd.read_csv('reit_news_collection/output.csv')

# Create a list to store individual company mentions
company_mentions = []

# Iterate through each row in the original DataFrame
for _, row in data.iterrows():
    # Extract companies from the 'companies_mentioned' column
    companies = str(row['companies_mentioned']).split(',')  # Convert to string
    
    # Calculate the sentiment score for each company and add to the list
    for company in companies:
        company_mentions.append({
            'month': row['month'],
            'year': row['year'],
            'company': company.strip(),  # Remove leading/trailing whitespace
            'sentiment_score': row['sentiment_score']
        })

# Create a new DataFrame from the list of dictionaries
company_mentions_df = pd.DataFrame(company_mentions)

# Convert 'month' and 'year' columns to integers
company_mentions_df['month'] = company_mentions_df['month'].astype(int)
company_mentions_df['year'] = company_mentions_df['year'].astype(int)

# Calculate the average sentiment scores per month, year, company
average_scores = company_mentions_df.groupby(['month', 'year', 'company']).mean().round(4)

# Reset the index for better readability
average_scores.reset_index(inplace=True)



# Print the resulting DataFrame
print(average_scores)
average_scores.to_csv('average_sentiment_scores.csv', index=False)

print(average_scores.describe())

      month  year                    company  sentiment_score
0         1    12                        nan           0.3846
1         1    13                        nan           0.9015
2         1    14                        nan           0.9390
3         1    15                        nan           0.5936
4         1    16  Armada Hoffler Properties           0.9601
...     ...   ...                        ...              ...
1651     12    22             Tanger Outlets           0.9287
1652     12    22                Timberlands           0.9790
1653     12    22             UMH Properties           0.9617
1654     12    22               Weyerhaeuser           0.9790
1655     12    22                        nan           0.9332

[1656 rows x 4 columns]
             month        year  sentiment_score
count  1656.000000  1656.00000      1656.000000
mean      5.935990    20.21558         0.864216
std       3.431788     2.51209         0.218899
min       1.000000    12.00000        -